In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
#credit for this codes
# - https://www.hackerearth.com/fr/practice/notes/praveen97uma/crawling-a-website-that-loads-content-using-javascript-with-selenium-webdriver-in-python/
# - https://github.com/LenaNevel/CAPSTONE/blob/master/00_getting_slugs.ipynb

## 0. Category별 상품 링크 크롤링

In [259]:
# chromedriver = "/usr/local/Cellar/chromedriver"
# driver = webdriver.Chrome(chromedriver)

## Using Headless Chrome version

chromedriver = "/usr/local/Cellar/chromedriver"
headless_options = webdriver.ChromeOptions()
headless_options.add_argument('headless')
headless_options.add_argument('window-size=1920x1080')
headless_options.add_argument('disable-gpu')
driver = webdriver.Chrome(chromedriver, options=headless_options) 

In [241]:
## select categories & create an empty dataframe
categories = ['skin-care-solutions', #702
              'moisturizing-cream-oils-mists', #806
              'cleanser', #495
              'facial-treatments' #582
              'eye-treatment-dark-circle-treatment', #221
              'face-mask', #305
              'sunscreen-sun-protection', #184
              'lip-treatments'] #166

#creating an empty data frame that all the URLs and categories will go in to
final_df = pd.DataFrame(columns = ['category', 'URL'])

for category in categories: #looping through the different categories

    print(category)
    
    page_num = 1 #the first page

    while True:

        #when running the following .get request in order for everything to work properly
        #the window needs to be popped open on the screen
        url = 'http://www.sephora.com/shop/'+ category + '?currentPage=' + str(page_num)
        driver.get(url)
        time.sleep(10)

        #check to see if the page is empty   
        try: 
            if driver.find_element_by_class_name('css-1kv743p').is_displayed():
                break

        except:

            #check to see if there is a pop up window1
            try:
                #exit the pop up window
                xpath = '//*[@id="modal0Dialog"]/button'
                btn = driver.find_element_by_xpath(xpath)
                btn.click()
                time.sleep(4)
            except:
                pass    

            #check to see if there is a pop up window2 (please_sign_in_to_sephora)
            try:
                xpath = '//*[@id="modal1Dialog"]/button'
                btn = driver.find_element_by_xpath(xpath)
                btn.click()
                time.sleep(4)
            except:
                pass    


            # Scroll down the page 10 times
            body = driver.find_element_by_css_selector('body')
            for i in range(10):
                body.send_keys(Keys.PAGE_DOWN)
                time.sleep(1)

            print('page', page_num, 'completely loaded')

            time.sleep(5)
            slug = driver.find_elements_by_class_name('css-ix8km1') #look for the urls of products    

            
            #from the list of URLs in slug pull all the href and make a dictionary with it and the category name            
            slugURL = list()

            for a in slug:
                subURL = {}
                subURL['category'] = category
                subURL['URL'] = a.get_attribute('href')
                slugURL.append(subURL)

            print('slug :', len(slug), 'slugURL :', len(slugURL))

            #append our data frame with categories and URLs
            df = pd.DataFrame(slugURL)

            print('page', page_num, 'done')

            #adding to go to next page
            page_num += 1
            #concatenating to get all in same df
            final_df = pd.concat([final_df, df], axis = 0, ignore_index = True) 

#close the chrome
driver.close()
print('Work Done')

cleanser
page 1 completely loaded
slug : 60 slugURL : 60
page 2 done
page 2 completely loaded
slug : 60 slugURL : 60
page 3 done
page 3 completely loaded
slug : 60 slugURL : 60
page 4 done
page 4 completely loaded
slug : 60 slugURL : 60
page 5 done
page 5 completely loaded
slug : 60 slugURL : 60
page 6 done
page 6 completely loaded
slug : 60 slugURL : 60
page 7 done
page 7 completely loaded
slug : 60 slugURL : 60
page 8 done
page 8 completely loaded
slug : 60 slugURL : 60
page 9 done
page 9 completely loaded
slug : 15 slugURL : 15
page 10 done
lip-treatments
page 1 completely loaded
slug : 60 slugURL : 60
page 2 done
page 2 completely loaded
slug : 60 slugURL : 60
page 3 done
page 3 completely loaded
slug : 46 slugURL : 46
page 4 done
Work Done


In [261]:
final_df.groupby('category').size()

category
eye-treatment-dark-circle-treatment    221
face-mask                              305
facial-treatments                      582
sunscreen-sun-protection               183
dtype: int64

In [264]:
#saving the final dataframe of URLs in to a csv folder
final_df.to_csv('./sephora_product_urls.csv', index = False)

In [265]:
## Categories
# - df1 = 'Cleanser', 'lip-treatments'
# - df2 = 'skin-care-solutions','moisturizing-cream-oils-mists'
# - df3 = 'facial-treatments','eye-treatment-dark-circle-treatment','face-mask','sunscreen-sun-protection'